# 14.32 final paper

## Setup

In [1]:
import pandas as pd
import numpy as np

## Load data

In [2]:
closures_raw = pd.read_csv('https://raw.githubusercontent.com/Keystone-Strategy/covid19-intervention-data/a41e4f282fab31793d01923c7d7445e45bddbbe0/complete_npis_inherited_policies.csv')

## Preprocess

Limit to school closure dates and necessary fields.

In [3]:
closures = closures_raw[
    (closures_raw.npi == 'school_closure') & ~closures_raw.county.isna()][
    ['fip_code', 'start_date']]

In [4]:
closures.start_date = pd.to_datetime(closures.start_date)